# 广播语义(Broadcasting semantics)
`作者：Tina`
`时间：2018-05-07`

许多PyTorch操作支持[NumPy Broadcasting Semantics](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html#module-numpy.doc.broadcasting)。

简而言之，如果一个PyTorch操作支持广播，那么它的Tensor参数可以被自动扩展为相同尺寸而不需要复制数据。

## 一般语义
如果下面的规则保持不变，两个张量是“可扩展的”(broadcastable)：
- 每个张量至少有一个维度。
- 当在维度大小上迭代时，从后面的维度开始，维度大小必须是相等的，其中一个要么是1，要么不存在。


In [12]:
import torch
x=torch.empty(5,7,3)
y=torch.empty(5,7,3)
#维度相同，因此两个张量是可扩展的 (i.e. the above rules always hold0)

x=torch.empty((0,))
y=torch.empty(2,2)
#x和y不是可扩展的， 因为不满足第二条规则

# 可以排列尾部维度
x=torch.empty(5,3,4,1)
y=torch.empty( 3,1,1)  #y.size()->(1,3,1,1)
# x和y是可扩展的。
# 倒数第一个维度：都是1
# 倒数第二个维度：y是1
# 倒数第三个维度： x size == y size
# 倒数第四个维度：y的维度不存在

# 但是
x=torch.empty(5,2,4,1)
y=torch.empty(  3,1,1)
#x和y是不可扩展的，因为第三个尾部维度：2 != 3

如果两个张量x，y是“可扩展的”，得到的张量大小是这样计算的：
- 如果x和y的维数不相等，那么在张量的维数中，在维度较少的张量的维度前置1，使它们相等。
- 然后，对于每个维度大小，产生的维度大小是x和y在该维度上的最大值。

例如：

In [13]:
# 可以排列尾部维度使阅读更容易
x = torch.empty(5,1,4,1)
y = torch.empty(  3,1,1)
print((x+y).size())

# 但并不是必要的：
x=torch.empty(1)
y=torch.empty(3,1,7)
print((x+y).size())

# 下面是一个不可扩展的例子
x=torch.empty(5,2,4,1)
y=torch.empty(3,1,1) # y.size()->(1,3,1,1)倒数第三个维度不满足规则，不相等，但是又不是1
#print((x+y).size())

torch.Size([5, 3, 4, 1])
torch.Size([3, 1, 7])


## 原地语义(In-place semantics)
原地操作不允许由于扩展而改变形状。

比如：

In [14]:
x=torch.empty(5,3,4,1)
y=torch.empty(3,1,1)
print((x.add_(y)).size()) #y的维度被扩展到和x一样了

# 但是：
x=torch.empty(1,3,1)
y=torch.empty(3,1,7)
#print((x.add_(y)).size()) # y的维度和x维度扩展相互影响

torch.Size([5, 3, 4, 1])


## 向后兼容(Backwards compatibility)
只要每个张量中的元素个数是相等的，PyTorch的先前版本允许特定的点函数(pointwise functions)在具有不同形状的张量上执行,逐点运算通过将每个张量看成一维的情况来执行。

PyTorch现在支持广播，“一维”的逐点操作将会被弃用，并且会在张量有相同数量的元素但不能被扩展的情况下生成一个Python警告。

注意，在两个张量形状不同的情况下，广播的引入会导致向后不兼容的变化，但两张量是可扩展的，并且具有相同数量的元素。例如:

In [15]:
torch.add(torch.ones(4,1), torch.randn(4))

tensor([[ 1.7691,  1.1850, -0.3452,  1.4147],
        [ 1.7691,  1.1850, -0.3452,  1.4147],
        [ 1.7691,  1.1850, -0.3452,  1.4147],
        [ 1.7691,  1.1850, -0.3452,  1.4147]])

过去的版本中，这样的操作产生的是(4,1)的张量，但是现在版本产生的是(4,4)的张量。

为了帮助识别代码中可能由广播而引入的向后不兼容情况，你可以设置`torch.utils.backcompat.broadcast_warning.enabled = True`，在这种情况下，它将生成python警告。

In [18]:
torch.utils.backcompat.broadcast_warning.enabled=True
torch.add(torch.ones(4,1), torch.ones(4))

tensor([[ 2.,  2.,  2.,  2.],
        [ 2.,  2.,  2.,  2.],
        [ 2.,  2.,  2.,  2.],
        [ 2.,  2.,  2.,  2.]])